# Trajectory Prior Example

Example that uses a simple constant-velocity prior over a trajectory.


In [1]:
import numpy as np

np.set_printoptions(6, suppress=True)

from pylgmath import so3op, se3op, Transformation
from pysteam.trajectory import Time, TrajectoryInterface
from pysteam.problem import OptimizationProblem
from pysteam.solver import GaussNewtonSolver, DoglegGaussNewtonSolver
from pysteam.evaluable.se3 import SE3StateVar
from pysteam.evaluable.vspace import VSpaceStateVar

Options:


In [2]:
num_states = 2  # total number of states
T_vi_init = Transformation()  # initial pose T_vi_init=T_ii (used as a prior)
w_iv_inv_init = np.array([[-1.0, 0., 0., 0., 0., 0.]]).T  # initial body-centric velocity (used as a prior)
dt = 1

Set up the trajectory that uses a constant velocity motion prior:


In [3]:
# states with initial conditions and associated timestamps
states = [(dt * i, Transformation(), 0.05 * i * np.ones((6, 1))) for i in range(num_states)]

# wrap states with corresponding steam state variables (no copying!)
state_vars = [(t, SE3StateVar(T_vi), VSpaceStateVar(w_iv_inv, locked=True)) for t, T_vi, w_iv_inv in states]

# construct a trajectory
Qc_inv = np.diag(1 / np.array([1.0, 0.001, 0.001, 0.001, 0.001, 0.001]))  # smoothing factor diagonal
traj = TrajectoryInterface(Qc_inv=Qc_inv)
for t, T_vi, w_iv_inv in state_vars:
  traj.add_knot(time=Time(t), T_k0=T_vi, w_0k_ink=w_iv_inv)

# add pose and velocity priors (associated with the initial timestamp t0)
traj.add_pose_prior(Time(states[0][0]), T_vi_init, np.eye(6))

Construct the optimization problem and solve:


In [4]:
opt_prob = OptimizationProblem()
opt_prob.add_state_var(*[v for state_var in state_vars for v in state_var[1:]])
opt_prob.add_cost_term(*traj.get_prior_cost_terms())

solver = DoglegGaussNewtonSolver(opt_prob, verbose=True)
solver.optimize()

Begin Optimization
------------------
Number of States:  2
Number of Cost Terms:  2
Initial Cost:  25.004999999999995
Iteration:    1  -  Cost:     6.2510  -  TR Shrink:  0.000  -  AvP Ratio:  1.000  -  Dogleg Segment: Gauss Newton   
Iteration:    2  -  Cost:     6.2510  -  TR Shrink:  0.000  -  AvP Ratio:  4.035  -  Dogleg Segment: Gauss Newton   
Termination Cause:  CONVERGED ABSOLUTE CHANGE
Total Optimization Time: 0.0165 seconds


Print out results:


In [5]:
print("First Pose:                 \n", states[0][1])
print("Second Pose:                \n", states[1][1])
print("Last Pose (full circle):    \n", states[-1][1])
print("First Vel:                  \n", states[0][2].T)
print("Second Vel:                 \n", states[1][2].T)
print("Last Vel:                   \n", states[-1][2].T)
print("Interp. Vel (t=t0+0.5*dT):  \n",
      traj.get_velocity_interpolator(Time(states[0][0]) + Time(secs=0.5 * dt)).evaluate().T)


First Pose:                 
 [[ 1.  0.  0. -0.]
 [ 0.  1.  0. -0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]
Second Pose:                
 [[ 0.999375 -0.024786  0.025203  0.022652]
 [ 0.025408  0.999373 -0.024677  0.024835]
 [-0.024576  0.025302  0.999378  0.02516 ]
 [ 0.        0.        0.        1.      ]]
Last Pose (full circle):    
 [[ 0.999375 -0.024786  0.025203  0.022652]
 [ 0.025408  0.999373 -0.024677  0.024835]
 [-0.024576  0.025302  0.999378  0.02516 ]
 [ 0.        0.        0.        1.      ]]
First Vel:                  
 [[0. 0. 0. 0. 0. 0.]]
Second Vel:                 
 [[0.05 0.05 0.05 0.05 0.05 0.05]]
Last Vel:                   
 [[0.05 0.05 0.05 0.05 0.05 0.05]]
Interp. Vel (t=t0+0.5*dT):  
 [[0.021473 0.024817 0.025181 0.024995 0.024847 0.025158]]
